### word2vec

- 详细介绍请参考：

    - https://zhuanlan.zhihu.com/p/26306795
    
    - 《word2vec 中的数学原理详解》
    
    - https://www.cnblogs.com/iloveai/p/word2vec.html


1. 为什么需要word2vec？

one-hot编码维度很大，而且不能在语义层面上对词进行表达，所以我们考虑采用一个新的语言模型。

2. 什么是word2vec？

word2vec是一种神经网络语言模型，将词映射到一个低纬稠密的词嵌入向量，常用的是CBOW和Skip-gram模型。这个模型的输入是词的one-hot encoder，模型的输出是softmax，但其实最后的输出是什么我们并不关心，我们关心的是模型第一个隐含层中的参数权重，这个参数权重就是我们需要的词嵌入矩阵，根据词嵌入矩阵乘以某个词的one hot向量，我们就可以得到该词的词向量。

3. 如何训练word2vec？

    <img src="./imgs/img03.png" width = "450" height = "450" />

    这里说明一点，隐层的激活函数其实是线性的，所以word2vec可以理解为线性模型。这里我认为W和W'都可以作为我们的嵌入矩阵，一个每一行是一个嵌入向量，一个每一列是一个嵌入向量。

    3.1 CBOW
    
    用一个词的上下文作为输入，来预测这个词本身，则这个模型叫做CBOW模型
<img src="./imgs/img05.png" width = "250" height = "250" />

    3.2 skip-gram
    
    用一个词作为输入，来预测它周围的上下文，这个模型叫做skip-gram模型
<img src="./imgs/img04.png" width = "250" height = "250" />

<img src="./imgs/img02.png" width = "300" height = "300" />


### 加速softmax的方法

最后输出的是个维度很高的词概率向量，这样softmax分母进行加和操作会很慢，所以我们采取一些技巧加速这个求和过程。

1. 层次softmax，使用分级的分类树

本质是把 N 分类问题变成 log(N)次二分类

为了避免计算所有词的softmax概率，word2vec使用霍夫曼树代替从隐藏层到softmax的映射。该方法不用为了获得最后的概率和而评估神经网络中的N个输出节点，而只需要评估大约log2(W)个节点。层次softmax使用一种二叉树结构来表示词典里的所有词，V个词都是二叉树的叶子节点，V-1个非叶子节点。

2. 采用负采样

本质是预测总体类别的一个子集

原来的训练过程是，每次用一个训练样本，则更新所有的权重，这样其实十分耗时的，其实我们不需要更新全部的参数，只需要更新那部分和这个样本相关的参数即可。负采样的意思是每次训练样本最后输出的值其实只有一个词是1，其他不是上下文范围内的词都是0，那我们可以从是0的单词中采样一部分如5~20个词来进行参数更新，而不是使用全部的词。在模型中的提现是，原来的输入x是词，输出也是词；现在的输入x是(词，词)输出为0或1。原来是一个巨大的10000维度的softmax，计算成本很高，现在是10000个二分类，而且每次迭代我们只需要训练它们其中的几个。